📦 Cell 1 – Imports & Setup

In [1]:
import json
import numpy as np
import os
import sys
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from src.utils.visualization import animate_dance, rotate_data, get_joint_color_config

CLUSTERS, CLUSTER_COLORS, JOINT_COLORS = get_joint_color_config()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


📂 Cell 2 – Load Data & Labels

In [2]:
chunk_path = "../data/processed/mariel_honey_chunks.npy"
metadata_path = "../data/processed/chunk_metadata.json"
label_path = "../data/labels/labels.json"

# 🔁 Load chunked data + metadata
chunks = np.load(chunk_path)  # shape: (N, T, J, D)
with open(metadata_path) as f:
    metadata = json.load(f)

# 📄 Load or create labels file
if os.path.exists(label_path):
    with open(label_path, "r") as f:
        labels = json.load(f)
else:
    labels = {}

💾 Cell 3 – Save Function

In [3]:
def save_labels():
    with open(label_path, "w") as f:
        json.dump(labels, f, indent=4)
    print("✅ Labels saved!")

🧠 Cell 4 – Define Widgets & Layout

In [4]:
chunk_selector = widgets.IntSlider(value=0, min=0, max=len(chunks) - 1, description='Chunk:')
prev_button = widgets.Button(description="⏮️ Prev", layout=widgets.Layout(width="100px"))
next_button = widgets.Button(description="⏭️ Next", layout=widgets.Layout(width="100px"))
chunk_id_display = widgets.HTML(value="<b>Chunk ID:</b> N/A", layout=widgets.Layout(margin="10px 0"))
label_options = sorted([
    "idle", "walking", "running", "jumping", "turning", "bending", "twisting",
    "happy", "sad", "angry", "excited", "relaxed", "dramatic",
    "hiphop", "ballet", "contemporary", "jazz", "salsa", "freestyle", "robotic", "folk",
    "slow", "medium", "fast", "syncopated", "stillness",
    "solo", "duet", "floorwork", "arm focus", "leg focus", "spinning", "pose"
])
text_input = widgets.SelectMultiple(
    options=label_options,
    description="Labels:",
    layout=widgets.Layout(width="100%", height="200px"),
    style={'description_width': 'initial'}
)
save_button = widgets.Button(description="💾 Save Label", button_style="success")
feedback_output = widgets.Output()
animation_output = widgets.Output()

🔄 Cell 5 – Define Callback Logic

In [5]:
def update_display(change=None):
    index = chunk_selector.value
    chunk_id = metadata[index]["id"]
    current_label = labels.get(chunk_id, [])
    if isinstance(current_label, list):
        text_input.value = tuple(current_label)  # SelectMultiple uses tuples
    else:
        text_input.value = ()
    chunk_id_display.value = f"<b>Chunk ID:</b> {chunk_id}"

    with animation_output:
        clear_output(wait=True)
        chunk = chunks[index]

        if chunk.shape[-1] != 3:
            raise ValueError(f"Unexpected shape {chunk.shape}, expected last dim = 3")
        if chunk.shape[1] == 3 and chunk.shape[2] != 3:
            chunk = chunk.transpose(1, 0, 2)
        chunk = rotate_data(chunk)

        print(f"🕺 Visualizing: {chunk_id}")
        animate_dance(chunk, CLUSTERS, JOINT_COLORS)

def on_save_clicked(b):
    index = chunk_selector.value
    chunk_id = metadata[index]["id"]
    label_list = list(text_input.value)
    if label_list:
        labels[chunk_id] = label_list
        save_labels()
        with feedback_output:
            clear_output()
            print(f"✅ Saved labels for {chunk_id}: {label_list}")
    label_list = list(text_input.value)


def go_prev(b):
    if chunk_selector.value > chunk_selector.min:
        chunk_selector.value -= 1

def go_next(b):
    if chunk_selector.value < chunk_selector.max:
        chunk_selector.value += 1

🖼️ Cell 6 – Set Event Bindings & Display UI

In [6]:
chunk_selector.observe(update_display, names="value")
save_button.on_click(on_save_clicked)
prev_button.on_click(go_prev)
next_button.on_click(go_next)

ui = widgets.VBox([
    widgets.HTML(value="<h3>💃 Dance Chunk Labeling Tool</h3>"),
    widgets.HBox([prev_button, chunk_selector, next_button]),
    chunk_id_display,
    text_input,
    save_button,
    feedback_output,
    animation_output
])

display(ui)
update_display()
